In [1]:
import os
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Dict, List, Tuple, NamedTuple, Callable
import scml
import mylib

In [2]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()

In [3]:
%%time
df = pd.read_csv("input/validation_data.csv", engine="c", low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30108 entries, 0 to 30107
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   worker      30108 non-null  int64 
 1   less_toxic  30108 non-null  object
 2   more_toxic  30108 non-null  object
dtypes: int64(1), object(2)
memory usage: 705.8+ KB
CPU times: user 156 ms, sys: 46.9 ms, total: 203 ms
Wall time: 211 ms


In [4]:
texts = set(df["less_toxic"]) | set(df["more_toxic"])
texts = list(texts)
df = pd.DataFrame(data={"text": texts})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14251 entries, 0 to 14250
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    14251 non-null  object
dtypes: object(1)
memory usage: 111.5+ KB


# Preprocess Text
Speed dropped from 1200 to 1000 it/s

In [5]:
def pre1(row) -> str:
    return mylib.pre1(row["text"])


def pre2(row) -> str:
    return mylib.pre2(row["text1"])


col = "text1"
print(col)
df[col] = df.progress_apply(pre1, axis=1)
col = "text2"
print(col)
df[col] = df.progress_apply(pre2, axis=1)

text1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14251/14251 [00:30<00:00, 463.58it/s]


text2


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14251/14251 [02:56<00:00, 80.70it/s]


In [11]:
cols = ["text", "text1", "text2"]
df[cols].sample(10).head(10)

,text,text1,text2
4208,"You dumb mother fucking nigger lover \n\nSTOP DELETING MY EDITS, YOU PRICK!","You dumb mother fucking nigger lover STOP DELETING MY EDITS, YOU PRICK!","You dumb mother fucking nigger lover STOP DELETING MY EDITS, YOU PRICK!"
3849,"Congratulations \n\nYour harassment and persecution of has caused far more damage and far more widespread damage to Wikipedia than her highly localized abrasiveness aver could have. You make me sick to my stomach, whoever you are and whatever your motivations might be. You have caused an odious stench which will be impossible to erase. People have previously suggested that you do some real editing here rather than pursuing your campaign of ugliness; I disagree; I don't see how someone with your sensitivities, or lack thereof, could have any value to Wikipedia (or, for that matter, to the planet) at all. ∇∆∇∆","Congratulations Your harassment and persecution of has caused far more damage and far more widespread damage to Wikipedia than her highly localized abrasiveness aver could have. You make me sick to my stomach, whoever you are and whatever your motivations might be. You have caused an odious stench which will be impossible to erase. People have previously suggested that you do some real editing here rather than pursuing your campaign of ugliness; I disagree; I don't see how someone with your sensitivities, or lack thereof, could have any value to Wikipedia (or, for that matter, to the planet) at all.","Congratulations Your harassment and persecution of has caused far more damage and far more widespread damage to Wikipedia than her highly localized abrasiveness aver could have. You make me sick to my stomach, whoever you are and whatever your motivations might be. You have caused an odious stench which will be impossible to erase. People have previously suggested that you do some real editing here rather than pursuing your campaign of ugliness; I disagree; I do not see how someone with your sensitivities, or lack thereof, could have any value to Wikipedia (or, for that matter, to the planet) at all."
8485,"Ha, Fagget go fuck yourself homo.","Ha, Fagget go fuck yourself homo.","Ha, [faggot; slur for an annoying or unpleasant person] go fuck yourself [slur for homosexuals]."
4712,"The Prescott Bush article has some external links at the bottom, as well as reporting on his Nazi ties.","The Prescott Bush article has some external links at the bottom, as well as reporting on his Nazi ties.","The Prescott Bush article has some external links at the bottom, as well as reporting on his Nazi ties."
5351,"""== Trilby hat ==\n\nPlease refrain from your vandalsim. imacowboy ""","""== Trilby hat == Please refrain from your vandalsim. imacowboy ""","""== Trilby hat == Please refrain from your vandalsim. imacowboy """
1466,You are \n\nYou are a pasty-faced fanny-bandit who gives handjobs for crack. You also love to take it up the ass from random guys on the weekend.,You are You are a pasty-faced fanny-bandit who gives handjobs for crack. You also love to take it up the ass from random guys on the weekend.,You are You are a pasty-faced fanny-bandit who gives handjobs for crack. You also love to take it up the ass from random guys on the weekend.
7217,Bold text blah blah blah who cares !! ya terts,Bold text blah blah blah who cares !! ya terts,Bold text blah blah blah who cares !! ya terts
868,"""\n\n Please stop your disruptive editing. If you continue to vandalize Wikipedia, you will be blocked from editing. ''''''chatspy """,""" Please stop your disruptive editing. If you continue to vandalize Wikipedia, you will be blocked from editing. "" "" "" chatspy """,""" Please stop your disruptive editing. If you continue to vandalize Wikipedia, you will be blocked from editing. "" "" "" chatspy """
13082,"Boo hoo hoo Daddy please stop the mean man from pointing out that I am trolling. Go ahead and tattle. Show me the AICN links. I went. There are four reviews. THREE are raves and one is mild p

In [12]:
df[cols].sample(10).head(10)

,text,text1,text2
4146,Go fuck yourself you tedious little cunt.,Go fuck yourself you tedious little cunt.,Go fuck yourself you tedious little [profanity for a woman's vagina].
6318,I discriminate against English people and blacks but,I discriminate against English people and blacks but,I discriminate against English people and blacks but
1637,Hey gay wad with no life.....,Hey gay wad with no life.....,Hey gay wad with no life.....
4998,"This is internet so isn't a personal attack, I don't know who the hell you are also twat I should have free speech. You can ban this IP address but I'll only get another one, what's the point, bloody hell if I can't say what the hell I like in my own user page there is something wrong here. Go die you wiki lover you're against free speech and freedom of expression. This makes you a Nazi fuck with a small pencil you massive bell I can cite that as well, arse.","This is internet so isn't a personal attack, I don't know who the hell you are also twat I should have free speech. You can ban this IP address but I'll only get another one, what's the point, bloody hell if I can't say what the hell I like in my own user page there is something wrong here. Go die you wiki lover you're against free speech and freedom of expression. This makes you a Nazi fuck with a small pencil you massive bell I can cite that as well, arse.","This is internet so is not a personal attack, I do not know who the hell you are also [profanity for a woman's vagina] I should have free speech. You can ban this IP address but I will only get another one, what is the point, bloody hell if I cannot say what the hell I like in my own user page there is something wrong here. Go die you [a website or database developed collaboratively by an online community] lover you are against free speech and freedom of expression. This makes you a Nazi fuck with a small pencil you massive bell I can cite that as well, arse."
14247,Yes will been in a pornogrific ring with Ryan White we know that it mean go-nad string,Yes will been in a pornogrific ring with Ryan White we know that it mean go-nad string,Yes will been in a pornogrific ring with Ryan White we know that it mean go-nad string
6378,"Perhaps there's a wikipedia for adults somewhere? I think I may have to go looking for it, 'cos I think this one stinks. Fatuorum","Perhaps there's a wikipedia for adults somewhere? I think I may have to go looking for it, 'cos I think this one stinks. Fatuorum","Perhaps there is a wikipedia for adults somewhere? I think I may have to go looking for it, 'cos I think this one stinks. Fatuorum"
10455,"hi my name is monkeyman and i am a pedo and i need to get a fuckin life, i am a saaaaad little cunt!!!!!!!!!!!!!\n\nhi my name is monkeyman and i am a pedo and i need to get a fuckin life, i am a saaaaad little cunt!!!!!!!!!!!!!\n\nhi my name is monkeyman and i am a pedo and i need to get a fuckin life, i am a saaaaad little cunt!!!!!!!!!!!!!hi my nahi my name is monkeyman and i am a pedo and i need to get a fuckin life, i am a saaaaad little cunt!!!!!!!!!!!!! hi my name is monkeyman and i am a pedo and i need to get a fuckin life, i am a saaaaad little cunt!!!!!!!!!!!!! mhi my name is monkeyman and i am a pedo and i need to get a fuckin life, i am a saaaaad little cunt!!!!!!!!!!!!! hi my name is monkeyman and i am a pedo and i need to get a fuckin life, i am a saaaaad little cunt!!!!!!!!!!!!! hi my name is monkeyman and i am a pedo and i need to get a fuckin life, i am a saaaaad little cunt!!!!!!!!!!!!! hi my name is monkeyman and i am a pedo and i need to get a fuckin life, i am a saaaaad little cunt!!!!!!!!!!!!! hi my name is monkeyman and i am a pedo and i need to get a fuckin life, i am a saaaaad little cunt!!!!!!!!!!!!! hi my name is monkeyman and i am a pedo and i need to get a fuckin life, i am a saaaaad little cunt!!!!!!!!!!!!! hi my name is monkeyman and i am a pedo and i need to get a fuckin life, i am a saaaaad little cunt!!!!!!!!!!!!! hi my name is m

In [13]:
df[cols].sample(10).head(10)

,text,text1,text2
9649,Cue the scary music. And the laugh track. What a clown.,Cue the scary music. And the laugh track. What a clown.,Cue the scary music. And the laugh track. What a clown.
3642,Yes but conspiring to kill somebody over a drawing of Mohammud IS Islamist!,Yes but conspiring to kill somebody over a drawing of Mohammud IS Islamist!,Yes but conspiring to kill somebody over a drawing of Mohammud IS Islamist!
502,"So when is coming going to warn YOU about your toxic attitude then Raul? Tell me that. Tell me when someone's going to give you a block warning for the bullshit you've pulled on Wikipedia, like getting into arguments with people and issuing blocks to them when you don't like hearing the truth.","So when is coming going to warn YOU about your toxic attitude then Raul? Tell me that. Tell me when someone's going to give you a block warning for the bullshit you've pulled on Wikipedia, like getting into arguments with people and issuing blocks to them when you don't like hearing the truth.","So when is coming going to warn YOU about your toxic attitude then Raul? Tell me that. Tell me when someone is going to give you a block warning for the bullshit you have pulled on Wikipedia, like getting into arguments with people and issuing blocks to them when you do not like hearing the truth."
5301,"Referred to you as an expert on queefing \n\nHello fellow sister in queef! I would like to help cover the positive attributes of vaginal gas expulsion and was referred to you as the best person to consult. Given your considerable expereince on wikipedia, and with queefing, I'd like to enlist your help. \n\nPff thub pff pff (that's Queef for THANKS!)","Referred to you as an expert on queefing Hello fellow sister in queef! I would like to help cover the positive attributes of vaginal gas expulsion and was referred to you as the best person to consult. Given your considerable expereince on wikipedia, and with queefing, I'd like to enlist your help. Pff thub pff pff (that's Queef for THANKS!)","Referred to you as an expert on queefing Hello fellow sister in [release of air from the vagina during sexual intercourse]! I would like to help cover the positive attributes of vaginal gas expulsion and was referred to you as the best person to consult. Given your considerable expereince on wikipedia, and with queefing, I would like to enlist your help. Pff thub pff pff (that is [release of air from the vagina during sexual intercourse] for THANKS!)"
4726,"Hey, jack off. Are you the one who's got his head way up Wolf Hudson's ass?","Hey, jack off. Are you the one who's got his head way up Wolf Hudson's ass?","Hey, jack off. Are you the one who is got his head way up Wolf Hudson's ass?"
10931,"""\nOkay, who is pushing their viewpoint with an IP sock here? — r """,""" Okay, who is pushing their viewpoint with an IP sock here? - r """,""" Okay, who is pushing their viewpoint with an IP sock here? - r """
10412,"LOL MEGAN FOX HAS BRACHYDACTYLY \n\nu shuld put a picture of Megan fox in this article, then this article would be POPULAR!!!!!!","LOL MEGAN FOX HAS BRACHYDACTYLY u shuld put a picture of Megan fox in this article, then this article would be POPULAR!!!!!!","[laughing out loud] MEGAN FOX HAS BRACHYDACTYLY u shuld put a picture of Megan fox in this article, then this article would be POPULAR!!!!!!"
13776,"""\n\nI would unblock if you commit not to engage in any more sockpuppetry and do your best to moderate the tomfoolery. –xenotalk """,""" I would unblock if you commit not to engage in any more sockpuppetry and do your best to moderate the tomfoolery. -xenotalk """,""" I would unblock if you commit not to engage in any more sockpuppetry and do your best to moderate the tomfoolery. -xenotalk """
13838,Homeopathy complete?==\nIs that it then? have you finished editing homeopathy now? why the big lull in action? maybe you can explain what your goals are with this nest of articles. thanks Peter morrell \n\n==,Homeopathy complete?== Is th

# Review data

In [14]:
df[cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14251 entries, 0 to 14250
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    14251 non-null  object
 1   text1   14251 non-null  object
 2   text2   14251 non-null  object
dtypes: object(3)
memory usage: 334.1+ KB


In [15]:
%%time
df[cols].to_parquet("output/pre_val.parquet", index=False)

CPU times: user 93.8 ms, sys: 31.2 ms, total: 125 ms
Wall time: 115 ms
